In [16]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained(
    "./model/normal_model",
    quantization_config=quantization_config
)

from peft import get_peft_model, LoraConfig
lora_config = LoraConfig(
    r=8,  # 低ランクアダプターのランク
    lora_alpha=32,  # アダプターのスケール
    lora_dropout=0.1,  # ドロップアウト率
)

model = get_peft_model(model, lora_config)

In [7]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, AutoTokenizer
import re
from tqdm import tqdm
import torch
from torch.nn import functional as F
from torch.optim import AdamW
import matplotlib.pyplot as plt
from torch import nn

device='cuda'
# モデルの準備
model = AutoModelForCausalLM.from_pretrained("../model/QLoRA_distill_model")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

In [8]:
for name, param in model.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")


model.embed_tokens.weight: requires_grad=False
model.layers.0.self_attn.q_proj.base_layer.weight: requires_grad=False
model.layers.0.self_attn.q_proj.lora_A.default.weight: requires_grad=False
model.layers.0.self_attn.q_proj.lora_B.default.weight: requires_grad=False
model.layers.0.self_attn.k_proj.weight: requires_grad=False
model.layers.0.self_attn.v_proj.base_layer.weight: requires_grad=False
model.layers.0.self_attn.v_proj.lora_A.default.weight: requires_grad=False
model.layers.0.self_attn.v_proj.lora_B.default.weight: requires_grad=False
model.layers.0.self_attn.o_proj.weight: requires_grad=False
model.layers.0.mlp.gate_proj.weight: requires_grad=False
model.layers.0.mlp.up_proj.weight: requires_grad=False
model.layers.0.mlp.down_proj.weight: requires_grad=False
model.layers.0.input_layernorm.weight: requires_grad=False
model.layers.0.post_attention_layernorm.weight: requires_grad=False
model.layers.1.self_attn.q_proj.base_layer.weight: requires_grad=False
model.layers.1.self_attn

In [5]:
for name, param in model.named_parameters():
    if 'lora_A' in name or 'lora_B' in name:
        param.requires_grad = True